# TensorFlow 卷积神经网络实现

本教程详细介绍卷积神经网络(CNN)的核心概念与实现，包括：
- 卷积运算的数学原理与可视化
- 使用低级API手动实现卷积
- 使用Keras构建完整的CNN分类器

## 环境要求
- TensorFlow >= 2.10
- NumPy
- Matplotlib
- scikit-learn (用于示例图像)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

# 设置随机种子确保结果可复现
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print(f"TensorFlow版本: {tf.__version__}")
print(f"GPU可用: {tf.config.list_physical_devices('GPU')}")

## 第一部分：理解卷积运算

### 1.1 卷积的核心思想

卷积神经网络的核心优势：
1. **参数共享** - 同一滤波器扫描整张图像，大幅减少参数量
2. **局部连接** - 每个神经元只关注输入的局部区域（感受野）
3. **平移等变性** - 特征无论出现在图像何处都能被检测到

In [ ]:
# 加载示例图像
from sklearn.datasets import load_sample_image

# 加载scikit-learn自带的示例图像并归一化到[0,1]
china = load_sample_image("china.jpg") / 255.0
flower = load_sample_image("flower.jpg") / 255.0

# 构建批次数据 (batch_size, height, width, channels)
images = np.array([china, flower], dtype=np.float32)
batch_size, height, width, channels = images.shape

print(f"图像批次形状: {images.shape}")
print(f"批次大小: {batch_size}, 高度: {height}, 宽度: {width}, 通道数: {channels}")

### 1.2 手动创建边缘检测滤波器

滤波器（卷积核）是特征检测器：
- 浅层滤波器检测边缘、角点等低级特征
- 深层滤波器组合低级特征形成高级语义特征

滤波器维度: `[filter_height, filter_width, input_channels, output_channels]`

In [ ]:
# 创建两个7x7的边缘检测滤波器
# 滤波器形状: [高度, 宽度, 输入通道数, 滤波器数量]
filters = np.zeros(shape=(7, 7, channels, 2), dtype=np.float32)

# 第一个滤波器：垂直边缘检测（中间列为1）
filters[:, 3, :, 0] = 1

# 第二个滤波器：水平边缘检测（中间行为1）
filters[3, :, :, 1] = 1

print(f"滤波器形状: {filters.shape}")
print("垂直滤波器(第4列):")
print(filters[:, :, 0, 0])

In [ ]:
# 使用tf.nn.conv2d执行卷积运算
# strides: [batch_stride, height_stride, width_stride, channel_stride]
# padding: 'SAME'保持输出尺寸与输入相同，'VALID'不填充
outputs = tf.nn.conv2d(
    input=images,
    filters=filters,
    strides=[1, 1, 1, 1],
    padding='SAME'
)

print(f"输出特征图形状: {outputs.shape}")

In [ ]:
# 可视化卷积结果
fig, axes = plt.subplots(2, 3, figsize=(14, 8))

for i, (img, title) in enumerate([(china, 'China'), (flower, 'Flower')]):
    axes[i, 0].imshow(img)
    axes[i, 0].set_title(f'{title} - 原图')
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(outputs[i, :, :, 0], cmap='gray')
    axes[i, 1].set_title(f'{title} - 垂直边缘')
    axes[i, 1].axis('off')
    
    axes[i, 2].imshow(outputs[i, :, :, 1], cmap='gray')
    axes[i, 2].set_title(f'{title} - 水平边缘')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

## 第二部分：使用Keras构建卷积层

### 2.1 Conv2D层详解

关键参数说明：
- `filters`: 滤波器数量，决定输出特征图深度
- `kernel_size`: 滤波器空间尺寸，通常使用3x3或5x5
- `strides`: 步幅，控制滤波器移动步长
- `padding`: 填充方式，'same'保持尺寸，'valid'不填充
- `activation`: 激活函数，通常使用ReLU

In [ ]:
# 使用Keras创建卷积层
conv_layer = keras.layers.Conv2D(
    filters=32,           # 32个滤波器
    kernel_size=3,        # 3x3卷积核
    strides=1,            # 步幅为1
    padding='same',       # 保持空间尺寸
    activation='relu',    # ReLU激活
    kernel_initializer='he_normal'  # He初始化，适合ReLU
)

# 计算参数量: (kernel_h * kernel_w * input_channels + 1) * filters
# 对于3x3卷积，3通道输入，32个滤波器: (3*3*3 + 1) * 32 = 896
print("卷积层配置完成")

## 第三部分：构建Fashion MNIST分类器

### 3.1 数据准备

Fashion MNIST数据集包含10类服装图像，每张28x28灰度图

In [ ]:
# 加载Fashion MNIST数据集
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

# 数据预处理
# 1. 添加通道维度: (samples, 28, 28) -> (samples, 28, 28, 1)
# 2. 归一化像素值到[0, 1]
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# 类别名称映射
CLASS_NAMES = [
    'T恤', '裤子', '套头衫', '裙子', '外套',
    '凉鞋', '衬衫', '运动鞋', '包', '踝靴'
]

print(f"训练集: {X_train.shape}, 标签: {y_train.shape}")
print(f"测试集: {X_test.shape}, 标签: {y_test.shape}")

In [ ]:
# 可视化部分样本
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_train[i].squeeze(), cmap='gray')
    ax.set_title(CLASS_NAMES[y_train[i]])
    ax.axis('off')
plt.tight_layout()
plt.show()

### 3.2 构建CNN模型

网络架构设计原则：
1. 随着网络加深，逐步增加滤波器数量（64 -> 128 -> 256）
2. 使用MaxPooling减少空间维度，降低计算量
3. 使用Dropout防止过拟合
4. 全连接层逐步收缩到输出类别数

In [ ]:
def build_cnn_model(input_shape=(28, 28, 1), num_classes=10):
    """
    构建用于图像分类的卷积神经网络
    
    架构: Conv->Pool->Conv->Conv->Pool->Conv->Conv->Pool->FC->FC->Output
    
    Parameters
    ----------
    input_shape : tuple
        输入图像形状 (height, width, channels)
    num_classes : int
        分类类别数
        
    Returns
    -------
    keras.Model
        编译好的CNN模型
    """
    model = keras.Sequential([
        # 第一卷积块: 64个滤波器
        keras.layers.Conv2D(
            64, kernel_size=3, padding='same', activation='relu',
            kernel_initializer='he_normal', input_shape=input_shape
        ),
        keras.layers.MaxPooling2D(pool_size=2),
        
        # 第二卷积块: 128个滤波器
        keras.layers.Conv2D(
            128, kernel_size=3, padding='same', activation='relu',
            kernel_initializer='he_normal'
        ),
        keras.layers.Conv2D(
            128, kernel_size=3, padding='same', activation='relu',
            kernel_initializer='he_normal'
        ),
        keras.layers.MaxPooling2D(pool_size=2),
        
        # 第三卷积块: 256个滤波器
        keras.layers.Conv2D(
            256, kernel_size=3, padding='same', activation='relu',
            kernel_initializer='he_normal'
        ),
        keras.layers.Conv2D(
            256, kernel_size=3, padding='same', activation='relu',
            kernel_initializer='he_normal'
        ),
        keras.layers.MaxPooling2D(pool_size=2),
        
        # 全连接分类器
        keras.layers.Flatten(),
        keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# 构建模型
model = build_cnn_model()
model.summary()

### 3.3 编译与训练

训练配置说明：
- **优化器**: Adam，自适应学习率优化器
- **损失函数**: sparse_categorical_crossentropy，适用于整数标签
- **评估指标**: accuracy，分类准确率

In [ ]:
# 编译模型
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 训练参数
EPOCHS = 10
BATCH_SIZE = 64

# 训练模型
history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_test, y_test),
    verbose=1
)

In [ ]:
# 可视化训练过程
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 损失曲线
axes[0].plot(history.history['loss'], label='训练损失')
axes[0].plot(history.history['val_loss'], label='验证损失')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('损失曲线')
axes[0].legend()
axes[0].grid(True)

# 准确率曲线
axes[1].plot(history.history['accuracy'], label='训练准确率')
axes[1].plot(history.history['val_accuracy'], label='验证准确率')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('准确率曲线')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

### 3.4 模型评估

In [ ]:
# 在测试集上评估
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"测试集损失: {test_loss:.4f}")
print(f"测试集准确率: {test_accuracy:.4f}")

In [ ]:
# 预测并可视化结果
predictions = model.predict(X_test[:16])
predicted_classes = np.argmax(predictions, axis=1)

fig, axes = plt.subplots(4, 4, figsize=(10, 10))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_test[i].squeeze(), cmap='gray')
    true_label = CLASS_NAMES[y_test[i]]
    pred_label = CLASS_NAMES[predicted_classes[i]]
    color = 'green' if y_test[i] == predicted_classes[i] else 'red'
    ax.set_title(f'真实:{true_label}\n预测:{pred_label}', color=color, fontsize=9)
    ax.axis('off')
plt.tight_layout()
plt.show()

## 总结

### 关键要点

1. **卷积层**通过参数共享和局部连接大幅减少参数量
2. **池化层**降低空间维度，增加平移不变性
3. **Dropout**在全连接层中防止过拟合
4. 网络深度增加时应相应增加滤波器数量

### 进一步学习

- 数据增强技术
- 批量归一化(Batch Normalization)
- 残差连接(ResNet)
- 迁移学习